# AFM Digital Twin: Imperfect Probe Example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)


## Task Overview

Participants are invited to develop methods for recovering the **"true" image** from SPM scans affected by probe artifacts, such as variable probe sizes and double tips. 

---

### **Key Question**

> **Given a corrupted SPM image affected by probe artifacts, can we automatically detect these distortions and accurately reconstruct the original, undistorted image?**


# 1. Run server

In [ ]:
!pip install pyro5
!pip install scifireaders
!pip install sidpy
!pip install pynsid
!pip install git+https://github.com/pycroscopy/DTMicroscope.git

In [ ]:
!run_server_afm

# 2. Client side

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import Pyro5.api
from IPython.display import clear_output, display

## 2.1 Get data

In [ ]:
!wget https://github.com/pycroscopy/DTMicroscope/raw/boris_dev/DTMicroscope/test/datasets/dset_spm1.h5

## 2.2 Register Data to the Digital twin

In [ ]:
uri = "PYRO:microscope.server@localhost:9092" #port for the AFM DT 9092
mic_server = Pyro5.api.Proxy(uri)
import sys

if 'google.colab' in sys.modules:
    mic_server.initialize_microscope("AFM", data_path = r"dset_spm1.h5")
else:
    mic_server.initialize_microscope("AFM", data_path = "../test/datasets/dset_spm1.h5")
mic_server.setup_microscope(data_source = 'Compound_Dataset_1')#upload dataset?
mic_server.get_dataset_info()

## Artefact 0: Blunt 'needle' with an offset tip

In [ ]:
#tip_radius
r_tip = np.random.uniform(0.2, 0.8)

#center (tip location)
center = [np.random.uniform(0.2, 0.8), np.random.uniform(0.2, 0.8)]

#kwargs for the blunt needle effect
kwargs = {'effect': 'real_tip', 'r_tip': r_tip, 'center': center}

#several effects can be combined in the mod_dict list
mod_dict = [{'effect': 'real_tip', 'kwargs': kwargs},]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7,3))

ax[0].imshow(dat[0].T, cmap='cividis', origin='lower')
ax[0].set_title('Ground truth\n(not available)')
ax[1].imshow(dat0[0].T, cmap='cividis', origin='lower')
ax[1].set_title('Corrupted image')

## Artefact 1: Tip doubling

In [ ]:
# r_tip: Sizes of both tips in a double-tip needle configuration
# center: Positions of each tip in the double-tip needle
# length_coef: Relative lengths of each tip in the double-tip needle, indicating proximity to the surface

kwargs = {'r_tip': [0.1, 0.05], 'center': [[0.2, 0.5], [0.6, 0.55]], 'length_coef': [1, 0.8]}

mod_dict = [{'effect': 'tip_doubling', 'kwargs': kwargs},]

The attribute direction specifies the fast scanning axis, which can be either horizontal or vertical.

In [ ]:
#ground truth scan topography
array_list, shape, dtype  =  mic_server.get_scan(channels=['HeightRetrace',], modification=None)
dat =  np.array(array_list, dtype=dtype).reshape(shape)

# Corrupted scan topography with the fast scan axis oriented horizontally
array_list, shape, dtype  =  mic_server.get_scan(channels=['HeightRetrace',], modification=mod_dict, direction='horizontal')
dat0 =  np.array(array_list, dtype=dtype).reshape(shape)

# Corrupted scan topography with the fast scan axis oriented vertically
array_list, shape, dtype  =  mic_server.get_scan(channels=['HeightRetrace',], modification=mod_dict, direction='vertical')
dat1 =  np.array(array_list, dtype=dtype).reshape(shape)
dat1.shape

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10,3))

ax[0].imshow(dat[0].T, cmap='cividis', origin='lower')
ax[0].set_title('Ground truth\n(not available)')
ax[1].imshow(dat0[0].T, cmap='cividis', origin='lower')
ax[1].set_title('Tip doubling\nfast axis: horizontal')
ax[2].imshow(dat1[0].T, cmap='cividis', origin='lower')
ax[2].set_title('Tip doubling\nfast axis: vertical')